In [1]:
import numpy as np

## jupyter notebook showing how to inject events with genie
This is fairly slow, and can be cpu intensive, so don't simulate more than 1,000 events like this. For large datasets use slurm with the py script

#### also the following cell was just to resolve some annoying import problems I got with vs code's conda environment

In [2]:
import os
import ctypes
import sys


modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

# library path
intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path

intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

# Python paths
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


In [3]:
from prometheus import Prometheus, config

In [4]:



import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final
from gevgen_injection import run_gevgen, convert_to_gst, convert_to_gtrac

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


#### the gevgen injection happens in file, gevgen_injection.py and is called by helper functions in cell below 

In [5]:

def main(num_events, simset=1):
    # timing
    start_time = time.time()


    print(f"Using simset: {simset}")
    processing_start_time = time.time()



    logger.info("Starting GENIE simulation")
    genie_start = time.time()
    ghep_file = run_gevgen(num_events, OUTPUT_DIR)### TODO: add extra parameters to this in addition to just num_events
    gtrac_file = convert_to_gtrac(ghep_file, OUTPUT_DIR)
    
    genie_time = time.time() - genie_start
    logger.info(f"GENIE simulation completed in {genie_time:.2f} seconds")
    prometheus_set, primary_set = parse_and_convert_genie(gtrac_file)
    processing_end_time = time.time()
    print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

    primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet' ## could make these unique if you don't want them to be overwritten
    prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

    # inject into cylinder:
    primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                        prometheus_set, ## child particle information
                        cylinder_radius=500, ## meters
                        cylinder_height = 1000, # meters
                        cylinder_center = (0, 0, 0), # meters not including offset
                        )

    primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)


    save_start_time = time.time()
    # position arrays to serialized strings
    prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
    num_events = len(prometheus_set)
    print(f"Processing {num_events} events")
    prometheus_set.to_parquet(prometheus_file_path)
    primary_set.to_parquet(primrary_file_path)
    save_end_time = time.time()
    print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

    ## GENIE stuff:
    config["injection"]["name"] = "GENIE"
    config["run"]["outfile"] = f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"
    config["run"]["nevents"] = num_events
    config["injection"]["GENIE"]["inject"] = False ## ideally this should toggle using gevgen and be true, but for now setting this to false helps solve a headache in promtheus.py
    config["injection"]["GENIE"]["simulation"] = {}
    ## geofile:
    config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
    ## ppc configuration - using the 'new' ppc:
    config['photon propagator']['name'] = 'PPC_UPGRADE'
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
    print(config["photon propagator"])
    config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = False ## just for printing, can turn to True if you don't want a cluttered print out!

    sim_start_time = time.time()
    p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
    p.sim()
    sim_end_time = time.time()
    print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")
    
    # end timing
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print('Finished without catastrophic error')
    return




In [6]:
num_events = 10 ## just a short dataset for testing
simset = 16
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")
print("Launching simulation")
main(num_events, simset)
print("Finished call")
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

2025-05-07 14:41:46,136 - __main__ - INFO - Starting GENIE simulation
2025-05-07 14:41:46,137 - gevgen_injection - INFO - Running GEVGEN with 10 events
2025-05-07 14:41:46,138 - gevgen_injection - INFO - genie command: ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '10', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '1,100', '-f', 'x^-2', '--seed', '12345', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-generator-list', 'Default', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


--------------------------------------------------------------
--------------------------------------------------------------
Launching simulation
Using simset: 16
cmd:  ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '10', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '1,100', '-f', 'x^-2', '--seed', '12345', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-generator-list', 'Default', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


2025-05-07 14:41:48,256 - gevgen_injection - INFO - GEVGEN completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root
2025-05-07 14:41:48,257 - gevgen_injection - INFO - Converting to GTRAC format using rootracker




*****************************************************************************************************************
*                                                                                                               *
*                                                   .oooooo.    oooooooooooo ooooo      ooo ooooo oooooooooooo  *
*                                                  d8P'  `Y8b   `888'     `8 `888b.     `8' `888' `888'     `8  *
*                                                 888            888          8 `88b.    8   888   888          * 
*                        Ndyooym          dN      888            888oooo8     8   `88b.  8   888   888oooo8     *
*                     Nds//+sdmoy       d+m       888     ooooo  888    "     8     `88b.8   888   888    "     * 
*                   Nh+//ohN  m+s      N//syyN    `88.    .88'   888       o  8       `888   888   888       o  *
*                 Ny+//od   Nh+oN       o///+      `Y8bood8P'   o888ooooood8 o8o    

2025-05-07 14:41:49,625 - gevgen_injection - INFO - GTRAC conversion completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:41:49,625 - __main__ - INFO - GENIE simulation completed in 3.49 seconds
2025-05-07 14:41:49,627 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:41:49,628 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:41:49,628 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:41:49,629 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:41:49,641 -

...

returning hadron 2212
returning hadron 2212


Configuring ppc in "./ppc_tmpdir16/"
Configuring nextgen sensors in "./ppc_tmpdir16/"
Configuring icemodel in "./ppc_tmpdir16/"
Configuring tiltmodel in "./ppc_tmpdir16/"
Configuring holeice from "./ppc_tmpdir16/as.dat"
Configured: xR=1 eff=1.015 sf=0.137 g=0.9
Ice anisotropy is k(130)=1,1,1
New Ice anisotropy is 1.82212,0.740818,0.740818
Renorm. NI anisotropy 1.69545,0.689318,0.689318
Ice absorption anisotropy scaling is 0
Initialized BFR diffusion patterns; s_eff=0.00246854 m^-1
Loaded 12 angsens coefficients
Loaded 131072 random multipliers
Loaded 5083 cable positions
Loaded 399 RDE coefficients
 Found 4761 OMs of type -1,0
 Found 399 OMs of type -1,1
Loaded 5160 DOMs (21x19)
Loaded 80x125 2d dust layer points
Ice layer thickness: UNIFORM
Loaded 42 wavelenth points
Loaded 42 RDE coefficients
Calculated value of ppm=2268.22
Loaded 171 ice layers
Air bubble parameters: 11.7 1350 1400

Processing f2k muons from stdin on device 0
HQUO(photons/max number of hits)=1
photons: 2350  hits: 0

returning hadron 2212
Handling charged pion/kaon 211


Configuring ppc in "./ppc_tmpdir16/"
Configuring nextgen sensors in "./ppc_tmpdir16/"
Configuring icemodel in "./ppc_tmpdir16/"
Configuring tiltmodel in "./ppc_tmpdir16/"
Configuring holeice from "./ppc_tmpdir16/as.dat"
Configured: xR=1 eff=1.015 sf=0.137 g=0.9
Ice anisotropy is k(130)=1,1,1
New Ice anisotropy is 1.82212,0.740818,0.740818
Renorm. NI anisotropy 1.69545,0.689318,0.689318
Ice absorption anisotropy scaling is 0
Initialized BFR diffusion patterns; s_eff=0.00246854 m^-1
Loaded 12 angsens coefficients
Loaded 131072 random multipliers
Loaded 5083 cable positions
Loaded 399 RDE coefficients
 Found 4761 OMs of type -1,0
 Found 399 OMs of type -1,1
Loaded 5160 DOMs (21x19)
Loaded 80x125 2d dust layer points
Ice layer thickness: UNIFORM
Loaded 42 wavelenth points
Loaded 42 RDE coefficients
Calculated value of ppm=2268.22
Loaded 171 ice layers
Air bubble parameters: 11.7 1350 1400

Processing f2k muons from stdin on device 0
HQUO(photons/max number of hits)=1
photons: 36  hits: 0
P

Handling charged pion/kaon -211
Prometheus simulation completed in 18.60 seconds
Total execution time: 22.39 seconds (0.37 minutes)
Finished without catastrophic error
I am melting.... AHHHHHH!!!!
Finished call
--------------------------------------------------------------
--------------------------------------------------------------


In [7]:

primary_set_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_set_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'
out_parquet =  f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"

prometheus_set = pd.read_parquet(prometheus_set_path)
primary_set = pd.read_parquet(primary_set_path)
out_parquet = pd.read_parquet(out_parquet)

### the position on the primary set and prometheus set don't have the ~1950m z offset btw

In [8]:
primary_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,True,4.120269,14,NC,2.134353,2.982149,-1,-1,-368.721036,88.456850,421.245561,"[-368.72103578864704, 88.45684952547404, 421.2...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[NC],QES;"
1,True,33.232259,14,NC,1.426824,6.156579,-1,-1,212.461672,434.854964,149.925236,"[212.46167245699337, 434.85496449183177, 149.9...",-1,nu:14;tgt:1000010010;N:2212;q:2(v);proc:Weak[N...
2,True,26.600404,14,CC,2.077803,5.532505,-1,-1,89.851211,382.540552,-360.937192,"[89.85121132316773, 382.5405518872392, -360.93...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
3,True,31.335799,14,CC,0.783648,4.757178,-1,-1,-71.863573,440.259533,-23.267043,"[-71.86357315244918, 440.2595330400279, -23.26...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
4,True,1.904728,14,CC,1.812305,3.623241,-1,-1,106.173808,218.684467,474.886431,"[106.17380790173436, 218.6844670480369, 474.88...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[CC],RES;..."
5,True,2.360738,14,CC,0.768298,6.217603,-1,-1,158.358657,204.666684,-397.937333,"[158.3586568004207, 204.66668388906854, -397.9...",-1,"nu:14;tgt:1000080160;N:2112;proc:Weak[CC],RES;..."
6,True,3.968209,14,CC,1.820722,4.941503,-1,-1,100.560792,-484.626704,268.056443,"[100.56079210824457, -484.6267042585643, 268.0...",-1,"nu:14;tgt:1000080160;N:2112;proc:Weak[CC],QES;"
7,True,36.513016,14,NC,1.024824,5.042005,-1,-1,163.178717,-413.261489,-214.533620,"[163.17871740628706, -413.2614885135935, -214....",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[N...
8,True,19.521958,14,CC,0.812751,5.935642,-1,-1,-207.864822,445.910324,59.351080,"[-207.8648217219488, 445.910323558575, 59.3510...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
9,True,2.421808,14,CC,1.428959,4.221690,-1,-1,101.967484,-196.176297,-161.305918,"[101.9674843904828, -196.17629748069425, -161....",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...


In [9]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False]","[0.00444, 3.9605257185188063, 1.05783024823171...","[22, 14, 2212, 2000000101]","[NC, NC, NC, NC]","[1.6575593216954645, 2.0394855167357426, 3.043...","[2.755463690177905, 2.958103217208648, 0.40732...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[-368.72103578864704, -368.72103578864704, -36...","[88.45684952547404, 88.45684952547404, 88.4568...","[421.2455614643902, 421.2455614643902, 421.245...","[[-368.72103578864704, 88.45684952547404, 421....","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
1,"[False, False, False, False, False, False]","[25.744678284252732, 1.5912865794041986, 5.187...","[14, 111, 2112, 111, 211, 111]","[NC, NC, NC, NC, NC, NC]","[1.4384030391588014, 1.404877901441277, 1.3568...","[-0.0963621052585267, -0.07814883702408411, -0...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[212.46167245699337, 212.46167245699337, 212.4...","[434.85496449183177, 434.85496449183177, 434.8...","[149.9252355369464, 149.9252355369464, 149.925...","[[212.46167245699337, 434.85496449183177, 149....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
2,"[False, False, False, False, False, False, Fal...","[14.705120183999083, 0.5559631923089461, 2.614...","[13, 111, 2112, 211, 111, 111, 2212, -211]","[CC, CC, CC, CC, CC, CC, CC, CC]","[2.0757049374486694, 2.0183306199295092, 2.071...","[-0.748282774086667, -0.582203690913106, -0.74...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[89.85121132316773, 89.85121132316773, 89.8512...","[382.5405518872392, 382.5405518872392, 382.540...","[-360.93719230603983, -360.93719230603983, -36...","[[89.85121132316773, 382.5405518872392, -360.9...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
3,"[False, False, False, False, False, False, False]","[19.904673998313992, 0.6045963532376314, 3.685...","[13, 211, 211, -211, 111, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC]","[0.7389864357426194, 0.793927515664132, 0.6975...","[-1.3681488189786675, -1.8664419360351454, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-71.86357315244918, -71.86357315244918, -71.8...","[440.2595330400279, 440.2595330400279, 440.259...","[-23.267042885691012, -23.267042885691012, -23...","[[-71.86357315244918, 440.2595330400279, -23.2...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
4,"[False, False, False, False]","[0.00385, 0.4985908788035815, 1.86633738306353...","[22, 13, 2212, 211]","[CC, CC, CC, CC]","[1.0182224070010635, 0.47839169703966467, 2.06...","[-2.8272527881868497, -1.4927317434681866, -2....","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[106.17380790173436, 106.17380790173436, 106.1...","[218.6844670480369, 218.6844670480369, 218.684...","[474.88643123575093, 474.88643123575093, 474.8...","[[106.17380790173436, 218.6844670480369, 474.8...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
5,"[False, False, False, False, False, False, False]","[0.00618, 1.0975641954166295, 1.12105649722162...","[22, 13, 2112, 2212, 2212, 211, -211]","[CC, CC, CC, CC, CC, CC, CC]","[1.5093383373311102, 0.8190975074433162, 1.643...","[1.996303249691743, 0.23405544680603085, 0.570...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[158.3586568004207, 158.3586568004207, 158.358...","[204.66668388906854, 204.66668388906854, 204.6...","[-397.9373333381465, -397.9373333381465, -397....","[[158.3586568004207, 204.66668388906854, -397....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
6,"[False, False, False, False, False, False, Fal...","[2.9087030454378104, 0.96783604951397

### and the normal mc_truth and photons fields from the parquet:


In [10]:
out_parquet

,mc_truth,photons
0,"{'interaction': 2, 'initial_state_energy': 4.1...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
1,"{'interaction': 2, 'initial_state_energy': 33....","{'sensor_pos_x': [224.58], 'sensor_pos_y': [43..."
2,"{'interaction': 1, 'initial_state_energy': 26....","{'sensor_pos_x': [174.47, 101.04], 'sensor_pos..."
3,"{'interaction': 1, 'initial_state_energy': 31....","{'sensor_pos_x': [-21.97], 'sensor_pos_y': [39..."
4,"{'interaction': 1, 'initial_state_energy': 1.9...","{'sensor_pos_x': [54.26], 'sensor_pos_y': [292..."
5,"{'interaction': 1, 'initial_state_energy': 2.3...","{'sensor_pos_x': [195.03], 'sensor_pos_y': [12..."
6,"{'interaction': 1, 'initial_state_energy': 3.9...","{'sensor_pos_x': [114.39, 158.94], 'sensor_pos..."
7,"{'interaction': 2, 'initial_state_energy': 36....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
8,"{'interaction': 1, 'initial_state_energy': 19....","{'sensor_pos_x': [-224.09, -145.45, -224.09, -..."
9,"{'interaction': 1, 'initial_state_energy': 2.4...","{'sensor_pos_x': [210.47, 46.29], 'sensor_pos_..."


In [11]:
out_parquet['mc_truth'][2]

{'interaction': 1,
 'initial_state_energy': 26.600404392730383,
 'initial_state_type': 14,
 'initial_state_zenith': 2.077803479928837,
 'initial_state_azimuth': -0.7506804902632054,
 'initial_state_x': 95.72204078053198,
 'initial_state_y': 380.02194336010433,
 'initial_state_z': -2332.912957809923,
 'final_state_energy': array([1.47051202e+01, 1.42527101e-02, 4.12656699e-02, 5.01399945e-02,
        5.55963192e-01, 2.61423463e+00, 7.62118454e-01, 7.30616473e+00,
        3.44948570e-01, 9.55530679e-01, 1.19694698e+00]),
 'final_state_type': array([  13.,   11.,   14.,  -12.,  111., 2112.,  211.,  111.,  111.,
        2212., -211.]),
 'final_state_zenith': array([2.07570494, 1.38551193, 2.13226019, 1.17465952, 2.01833062,
        2.07107462, 1.81318441, 2.06932305, 2.12027141, 2.02442431,
        1.94066615]),
 'final_state_azimuth': array([-0.74828277,  1.47899129,  2.21695426, -1.12969056, -0.58220369,
        -0.74339616, -0.92648401, -0.73127079,  1.47319115, -0.697488  ,
        -0.

In [12]:
out_parquet['photons'][2]

{'sensor_pos_x': array([174.47, 101.04]),
 'sensor_pos_y': array([315.54, 412.79]),
 'sensor_pos_z': array([-2422.53, -2345.75]),
 'string_id': array([65, 72]),
 'sensor_id': array([59, 54]),
 't': array([765.78894 , 286.842651]),
 'id_idx': array([1, 1])}

#### so I guess you don't need to load the prometheus and primary set if you have the mc_truth, but maybe something to keep in mind for one weight or anything needed to reweight to flux